<a href="https://colab.research.google.com/github/Ndeffoloic/RAGImplementation/blob/main/RobKerrAIGenerativeChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pinecone
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [14]:
#Base python data handling environment import
import pandas as pd
import os
from tqdm.auto import tqdm
import time


#pinecode is a cloud-based vector Database we will use to store embedding
import pinecone

#OpenAI is used for the embedding LLM and GenAI model used to generate responses.
import openai

#Langchain is middleware that ties together the components of the embedding and retrieval pipelines
#The embedding chain creates searchable vectors of our data

from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.vectorstores import Pinecone

#Using your API key, initialize your client connection to Pinecone
from pinecone import Pinecone, ServerlessSpec

#A link in the chain to operate a chat session
from langchain.chat_models import ChatOpenAI

#we will maintain some memory of the chat so follow-up questions will be context-sensitive
from langchain.chains.conversation.memory \
import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

#Access your secret keys in Python via :
from google.colab import userdata


# Get Environment Variables

In [15]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
EMBEDDING_MODEL="text-embedding-ada-002"
GENAI_MODEL='gpt-3.5-turbo'

PINECONE_KEY=userdata.get("PINECONE_KEY")
PINECONE_ENV=os.getenv("PINECONE_ENV")
PINECONE_INDEX_NAME="default" #this will be created below.

pc = Pinecone(PINECONE_KEY)

#Upsert data

In [18]:
#Create a serverless index

"""
An index defines the dimension of vectors to be stored and the similarity metric to be used when querying them.
Create a serverless index with a dimension and similarity metric based on the mebedding model you'll use to create the vector embeddings :
"""
index_name = "quickstart"
pc.create_index(
    name=index_name,
    dimension=1024,
    metric="cosine", #replace with your model metric
    spec=ServerlessSpec(
        cloud='aws',
        region="us-east-1"
    )
)



{
    "name": "quickstart",
    "metric": "cosine",
    "host": "quickstart-k1hmcva.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}

In [25]:
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
    {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
]
tel = {'jack': 4098, 'sape': 4139}
tel['guido'] = 4127
print(tel)

tel['jack']

del tel['sape']
tel['irv'] = 4127
tel

print(list(data[3]))

sorted(tel)

'guido' in tel

'jack' not in tel
print(type(data[0]))
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)
print(embeddings[0])

{'jack': 4098, 'sape': 4139, 'guido': 4127}
['jack', 'guido', 'irv']
<class 'dict'>
{'vector_type': dense, 'values': [0.04931640625, -0.01328277587890625, ..., -0.0196380615234375, -0.010955810546875]}


In [ ]:
#Fetch only context knowledge about Detroit and London
filtered_df = df.loc[df['subject'].isin(['Detroit','London'])]
print(filtered_df['Subject'].value_counts())
filtered_df.head()


NameError: name 'df' is not defined